In [1]:
from utz import *
from nj_crashes.paths import ROOT_DIR
from njsp import crashes
from njsp.paths import PROJECTED_CSV
from njsp.ytc import to_ytc
from njsp.ytd import oldest_commit_rundate_since, projected_roy_deaths
from njsp import Ytd

chdir(ROOT_DIR)

In [2]:
ytd = Ytd()
prv_rundate = ytd.prv_rundate
prv_rundate

Searching for oldest commit with rundate ≥2023-05-24


Found rundate 2023-05-23 10:00:02 < 2023-05-24 at commit daa0c976; returning commit e1e912d


'2023-05-24 10:00:02'

In [3]:
prv_ytd_ratio = ytd.cur_ytd_deaths / ytd.prv_ytd_deaths
pct_change = (prv_ytd_ratio - 1) * 100

print(f'As of {ytd.prv_rundate}, NJSP was reporting {ytd.prv_ytd_total} YTD deaths')
print(f'Current YTD Deaths ({ytd.rundate}): {ytd.cur_ytd_deaths}')
print(f'Previous year YTD Deaths (adjusted; {ytd.prv_rundate}): {ytd.prv_ytd_deaths}')
print(f'Projected {ytd.cur_year} total: {ytd.projected_year_total}')
print(f'{pct_change:.1f}% change')

As of 2023-05-24 10:00:02, NJSP was reporting 191 YTD deaths
Current YTD Deaths (2024-05-24 10:00:02-04:00): 241
Previous year YTD Deaths (adjusted; 2023-05-24 10:00:02): 191.80667293141798
Projected 2024 total: 699.2997722734619
25.6% change


FAUQStats cache miss: 2023, Wed May 24 10:00:02 EDT 2023


Find the oldest commit with rundate less than 1 year ago, load crashes from that time:

In [4]:
prv_crashes = ytd.prv_ytd_crashes
prv_crashes

,CCODE,CNAME,MCODE,MNAME,HIGHWAY,LOCATION,FATALITIES,FATAL_D,FATAL_P,FATAL_T,FATAL_B,INJURIES,STREET,dt
ACCID,,,,,,,,,,,,,,
12249,07,Essex,0714,Newark City,NaN,Mulberry St,1.0,0.0,0.0,1.0,0.0,NaN,Mulberry St,2023-01-01 01:49:00-05:00
12251,01,Atlantic,0112,Hamilton Twsp,322,State Highway 322 MP 46.4,1.0,1.0,0.0,0.0,0.0,0.0,NaN,2023-01-02 09:26:00-05:00
12253,12,Middlesex,1217,Piscataway Twsp,NaN,South Randolphville Rd,1.0,1.0,0.0,0.0,0.0,0.0,South Randolphville Rd,2023-01-02 19:41:00-05:00
12254,02,Bergen,0221,Garfield City,NaN,Outwater Ln,1.0,0.0,0.0,1.0,0.0,0.0,Outwater Ln,2023-01-03 20:52:00-05:00
12267,07,Essex,0714,Newark City,NaN,Washington St,1.0,0.0,0.0,1.0,0.0,NaN,Washington St,2023-01-04 06:21:00-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12461,05,Cape May,0506,Middle Twsp,654,County 654,1.0,0.0,0.0,1.0,0.0,NaN,NaN,2023-05-17 06:40:00-04:00
12462,16,Passaic,1608,Paterson City,NaN,Main St,1.0,1.0,0.0,0.0,0.0,NaN,Main St,2023-05-18 19:10:00-04:00
12465,08,Gloucester,0820,West Deptford Twsp,643,County 643 (Spur (County Routes Only)),1.0,1.0,0.0,0.0,0.0,0.0,NaN,2023-05-20 17:28:00-04:00


Load previous year YTD and final counts, and current year YTD counts:

In [5]:
prv_year = ytd.prv_year
cur_year = ytd.cur_year

prv_ytc = to_ytc(prv_crashes)
prv_ytd = prv_ytc.loc[prv_year]

prv_end = to_ytc(ytd.prv_end_crashes).loc[prv_year]

cur_ytc = to_ytc(ytd.cur_ytd_crashes)
cur_ytd = cur_ytc.loc[cur_year]

cur_ytd

,driver,passenger,pedestrian,cyclist,crashes
county,,,,,
Atlantic,9,1,7,2,19
Bergen,8,3,9,0,19
Burlington,14,3,2,0,19
Camden,8,2,9,0,19
Cape May,1,0,0,0,1
Cumberland,5,1,1,0,7
Essex,6,3,8,0,17
Gloucester,6,2,0,0,8
Hudson,2,2,1,0,4


Compute fraction of current year that has elapsed (year-to-date; "YTD") and still remains (rest-of-year; "ROY"):

In [6]:
cur_ytd_frac = ytd.cur_year_frac
cur_roy_frac = 1 - cur_ytd_frac
print('%.1f%% through the year, %.1f%% remaining' % (ytd.cur_year_frac * 100, ytd.cur_roy_frac * 100))

39.4% through the year, 60.6% remaining


Combine previous year YTD / end with current year YTD:

In [7]:
def melt(df, name):
    return (
        df
        .melt(ignore_index=False, var_name='type')
        .set_index('type', append=True)
        .value
        .rename(name)
    )

z = sxs(
    melt(prv_ytd, 'prv_ytd'),
    melt(prv_end, 'prv_end'),
    melt(cur_ytd, 'cur_ytd'),
).fillna(0).astype(int)
z

prv_ytd  prv_end  cur_ytd
county     type                                 
Atlantic   driver            6       17        9
Bergen     driver            7       20        8
Burlington driver            4       26       14
Camden     driver            6       18        8
Cape May   driver            2        4        1
...                        ...      ...      ...
Sussex     driver            0        6        1
           passenger         0        2        0
           pedestrian        0        1        1
           cyclist           0        0        0
           crashes           0        6        2

[105 rows x 3 columns]

In [8]:
def project_roy(r):
    return int(round(projected_roy_deaths(r.prv_ytd, r.prv_end, r.cur_ytd, cur_ytd_frac)))

roy = z.apply(project_roy, axis=1).rename('roy')
projected = (
    (z.cur_ytd + roy)
    .rename('projected')
    .reset_index(level=1)
    .pivot(columns='type', values='projected')
)
projected

type,crashes,cyclist,driver,passenger,pedestrian
county,,,,,
Atlantic,49,4,22,7,16
Bergen,43,1,22,7,15
Burlington,64,1,58,7,4
Camden,64,3,22,7,51
Cape May,4,0,3,1,1
Cumberland,20,0,15,3,3
Essex,56,1,18,7,31
Gloucester,27,1,19,5,2
Hudson,14,1,7,5,4


In [9]:
projected.to_csv(PROJECTED_CSV)